# Transfer Learning

Este notebook explora o conceito de Transfer Learning (Aprendizagem por Transferência), uma técnica em deep learning que consiste em reutilizar um modelo pré-treinado em uma nova tarefa. Em vez de treinar uma rede neural do zero, o que exige grandes volumes de dados e poder computacional, podemos aproveitar o conhecimento encapsulado em modelos que foram treinados em datasets massivos, como o ImageNet.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, utils
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import copy
from PIL import Image
import glob

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
import requests
import zipfile
from io import BytesIO

if not os.path.exists('data/hymenoptera_data'):
    url = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"
    response = requests.get(url)
    with zipfile.ZipFile(BytesIO(response.content)) as z:
        z.extractall("data/hymenoptera_data")
    print("Dataset 'hymenoptera_data' baixado e extraído.")
else:
    print("Dataset 'hymenoptera_data' já existe.")

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.class_names = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.class_names)}

        self.data = []
        for class_name in self.class_names:
            class_dir = os.path.join(self.root_dir, class_name)
            for filename in os.listdir(class_dir):
                if filename.endswith(('.png', '.jpg', '.jpeg')):
                    path = os.path.join(class_dir, filename)
                    item = (path, self.class_to_idx[class_name])
                    self.data.append(item)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        path, label = self.data[idx]
        image = Image.open(path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'data/hymenoptera_data'

# Criando instâncias de Dataset
train_dataset = CustomImageDataset(os.path.join(data_dir, 'train'), data_transforms['train'])
val_dataset = CustomImageDataset(os.path.join(data_dir, 'val'), data_transforms['val'])

# Criando instâncias de DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=4)

# Obtendo informações
class_names = train_dataset.class_names
train_size = len(train_dataset)
val_size = len(val_dataset)

print(f"Classes: {class_names}")
print(f"Tamanho do dataset de treino: {train_size}")
print(f"Tamanho do dataset de validação: {val_size}")

In [ ]:
def imshow(inp, title=None):
    """Função para exibir um tensor de imagem."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

# Obtém um batch de dados de treino
inputs, classes = next(iter(train_dataloader))

# Cria uma grade a partir do batch
out = utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

In [ ]:
from tqdm.notebook import tqdm

def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=25):
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')

        for phase, loader in [('train', train_loader), ('val', val_loader)]:
            model.train(phase == 'train')
            running_loss, running_corrects = 0.0, 0
            size = len(loader.dataset)

            loop = tqdm(loader, desc=f'{phase.capitalize()}')
            for inputs, labels in loop:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels)

            epoch_loss = running_loss / size
            epoch_acc = running_corrects.double() / size
            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc.item())

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    return history


def plot_history(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    ax1.plot(history['train_acc'], label='Train Accuracy')
    ax1.plot(history['val_acc'], label='Validation Accuracy')
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    
    ax2.plot(history['train_loss'], label='Train Loss')
    ax2.plot(history['val_loss'], label='Validation Loss')
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    
    plt.show()

## Treinamento de um Modelo Baseline

Para entender o efeito do transfer learning, primeiro treinaremos uma rede convolucional (CNN) simples a partir do zero. Dado o tamanho reduzido do nosso dataset, é altamente provável que este modelo sofra de overfitting, o que servirá como um ponto de referência.

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=len(class_names)):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Flatten(),
            nn.Linear(32 * 56 * 56, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.model(x)

baseline_model = SimpleCNN().to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer_baseline = optim.Adam(baseline_model.parameters(), lr=0.001)

history_baseline = train_model(
    baseline_model, criterion, optimizer_baseline, train_dataloader, val_dataloader, num_epochs=25
)

In [ ]:
plot_history(history_baseline)

## Modelos Pré-treinados

O `torchvision.models` oferece acesso a diversas arquiteturas de modelos já treinados no dataset ImageNet. Esses modelos aprenderam a extrair hierarquias ricas de características (features), que podem ser aproveitadas em outras tarefas.

In [ ]:
# Carregando um modelo ResNet-50 pré-treinado
weights = models.ResNet50_Weights.DEFAULT
resnet50_pretrained = models.resnet50(weights=weights)

# A última camada 'fc' (fully connected) tem 1000 saídas,
# correspondentes às 1000 classes do ImageNet.
print(resnet50_pretrained)

### Usando um Modelo Pré-treinado para Inferência

Um modelo pré-treinado pode ser usado diretamente para inferência. Basta carregá-lo, colocá-lo em modo de avaliação com `.eval()` e passar uma imagem pré-processada. A saída será um vetor de scores para as 1000 classes do ImageNet.

In [ ]:
url = "https://www.uni-jena.de/unijenamedia/387585/elefant.jpg?height=428&width=760"
response = requests.get(url, stream=True)
img = Image.open(response.raw)

plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
preprocess = data_transforms['val']
img_tensor = preprocess(img).unsqueeze(0)

print(preprocess)
print(img_tensor.shape)

In [ ]:
resnet50_pretrained.eval()
with torch.no_grad():
    output = resnet50_pretrained(img_tensor)

print(output.shape)

In [ ]:
probabilities = torch.nn.functional.softmax(output[0], dim=0)
top5_prob, top5_catid = torch.topk(probabilities, 5)

for i in range(top5_prob.size(0)):
    print(f"Classe: {weights.meta['categories'][top5_catid[i]]}, Probabilidade: {top5_prob[i].item():.4f}")

## Transfer Learning: Congelando Camadas (Freezing)

A estratégia de "feature extraction" consiste em congelar os pesos das camadas convolucionais de um modelo pré-treinado e substituir a camada de classificação final por uma nova, adequada ao nosso problema. Apenas os pesos dessa nova camada serão treinados, o que é computacionalmente eficiente e previne overfitting em datasets pequenos.

In [ ]:
from torchvision import models

model_conv = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

# Congelar todos os parâmetros da rede
for param in model_conv.parameters():
    param.requires_grad = False

print(model_conv)

In [ ]:
num_ftrs = model_conv.fc.in_features

# Substituir a camada 'fc' por uma nova camada Linear
# Os parâmetros desta nova camada terão `requires_grad=True` por padrão
model_conv.fc = nn.Linear(num_ftrs, len(class_names))

model_conv = model_conv.to(device)

print("Estrutura da última camada modificada:")
print(model_conv.fc)

## Treinando o Modelo com Transfer Learning

Agora, vamos treinar o modelo modificado. O otimizador Adam será configurado para atualizar apenas os parâmetros da nova camada de classificação. Esperamos ver uma convergência muito mais rápida e uma acurácia de validação significativamente maior em comparação com o modelo baseline.

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer_conv = optim.Adam(model_conv.fc.parameters(), lr=0.001)

# Treinar o modelo
history_conv = train_model(
    model_conv, criterion, optimizer_conv, train_dataloader, val_dataloader, num_epochs=5
)

In [ ]:
plot_history(history_conv)

## Exercícios

### Exercício 1

Descongele mais das últimas camadas (à sua escolha), por exemplo `model.layer4[1]`, e treine novamente o modelo.

### Exercício 2

Escolha outro modelo pré-treinado em [Torchvision Models](https://docs.pytorch.org/vision/main/models.html) e substitua no modelo. Lembre-se de alterar a última camada de classificação.